In [102]:
import pandas as pd
import os

In [117]:
area_df = pd.read_csv('../../data/4.etc/processed_area_add_df.csv')


In [118]:
crime_raw = pd.read_csv("../../data/1.raw/crime/11_경찰청_범죄 발생 지역별 통계_20231231.csv",encoding="cp949")
crime_raw.head()

,범죄대분류,범죄중분류,서울종로구,서울중구,서울용산구,서울성동구,서울광진구,서울동대문구,서울중랑구,서울성북구,...,외국러시아,외국튀르키예,외국중국,외국일본,외국필리핀,외국베트남,외국태국,외국말레이시아,외국인도네시아,외국기타국가
0,강력범죄,살인기수,2,0,1,1,0,2,4,1,...,0,0,0,0,1,0,0,0,0,0
1,강력범죄,살인미수등,1,3,2,1,2,3,2,0,...,1,0,1,0,0,0,0,0,0,1
2,강력범죄,강도,5,6,3,2,5,4,2,1,...,0,0,0,0,3,0,0,0,0,0
3,강력범죄,강간,25,30,45,22,42,36,40,25,...,0,0,3,1,3,1,1,1,0,3
4,강력범죄,유사강간,10,4,4,6,9,5,9,7,...,0,0,0,0,1,0,0,1,0,1


In [119]:
print("▶ area_df columns:", area_df.columns.tolist())
print("▶ crime_raw columns:", crime_raw.columns.tolist())

▶ area_df columns: ['시점', '행정구역(시)', '행정구역(시군구)별', '총인구수', '남자인구수', '여자인구수', '영아기', '아동기', '청소년기', '청년기', '중년기', '노년기', 'CCTV', '1인가구비율', '경찰서수']
▶ crime_raw columns: ['범죄대분류', '범죄중분류', '서울종로구', '서울중구', '서울용산구', '서울성동구', '서울광진구', '서울동대문구', '서울중랑구', '서울성북구', '서울강북구', '서울도봉구', '서울노원구', '서울은평구', '서울서대문구', '서울마포구', '서울양천구', '서울강서구', '서울구로구', '서울금천구', '서울영등포구', '서울동작구', '서울관악구', '서울서초구', '서울강남구', '서울송파구', '서울강동구', '부산중구', '부산서구', '부산동구', '부산영도구', '부산부산진구', '부산동래구', '부산남구', '부산북구', '부산강서구', '부산해운대구', '부산사하구', '부산금정구', '부산연제구', '부산수영구', '부산사상구', '부산기장군', '대구중구', '대구동구', '대구서구', '대구남구', '대구북구', '대구수성구', '대구달서구', '대구달성군', '대구군위군', '인천중구', '인천동구', '인천미추홀구', '인천연수구', '인천남동구', '인천부평구', '인천계양구', '인천서구', '인천강화군', '인천옹진군', '광주동구', '광주서구', '광주남구', '광주북구', '광주광산구', '대전동구', '대전중구', '대전서구', '대전유성구', '대전대덕구', '울산중구', '울산남구', '울산동구', '울산북구', '울산울주군', '세종시', '경기도고양시', '경기도과천시', '경기도광명시', '경기도광주시', '경기도구리시', '경기도군포시', '경기도김포시', '경기도남양주시', '경기도동두천시', '경기도부천시', '경기도성남시', '경기도수원시', '경기도시흥시', '경기도안산시', '경기도안성시',

In [120]:
# ----------------------------------------------------
# 1. 지역별 컬럼만 추출 + 숫자형으로 변환
#    - 앞 2개: '범죄대분류', '범죄중분류'
#    - 나머지: '서울종로구', '부산해운대구', ..., '제주제주시', '외국미국', ...
#    - 외국* 컬럼은 분석에서 제외
# ----------------------------------------------------
region_cols = [
    c for c in crime_raw.columns
    if c not in ["범죄대분류", "범죄중분류"] and not c.startswith("외국")
]

# 쉼표 제거 + 공백 제거 + int 변환
for col in region_cols:
    crime_raw[col] = (
        crime_raw[col]
        .astype(str)
        .str.replace(",", "", regex=False)
        .str.strip()
        .replace("", "0")
        .astype(int)
    )

# ----------------------------------------------------
# 2. 범죄종류(대분류/중분류)를 컬럼으로 가져오는 wide 테이블 만들기
#    - 지금: 행 = 범죄대분류/중분류, 열 = 지역
#    - 목표: 행 = 지역, 열 = 범죄중분류(또는 대분류_중분류)
# ----------------------------------------------------
# (범죄대분류, 범죄중분류)를 index로, 지역 컬럼들만 가져와서 전치(T)
crime_pivot = crime_raw.set_index(["범죄대분류", "범죄중분류"])[region_cols].T

# 현재 컬럼은 MultiIndex(범죄대분류, 범죄중분류) → 하나의 문자열로 합치기
crime_pivot.columns = [
    f"{major}_{minor}" for major, minor in crime_pivot.columns.to_list()
]

# index 이름을 '지역'으로 만들고 컬럼으로 올리기
crime_pivot.index.name = "지역"
crime_pivot = crime_pivot.reset_index()

print("▶ crime_pivot head:")
print(crime_pivot.head())

▶ crime_pivot head:
      지역  강력범죄_살인기수  강력범죄_살인미수등  강력범죄_강도  강력범죄_강간  강력범죄_유사강간  강력범죄_강제추행  \
0  서울종로구          2           1        5       25         10        204   
1   서울중구          0           3        6       30          4        182   
2  서울용산구          1           2        3       45          4        195   
3  서울성동구          1           1        2       22          6         92   
4  서울광진구          0           2        5       42          9        188   

   강력범죄_기타 강간 강제추행등  강력범죄_방화  절도범죄_절도범죄  ...  특별경제범죄_특별경제범죄  마약범죄_마약범죄  \
0                 0        4       1173  ...            523        100   
1                 0       13       1523  ...            727         73   
2                 0       16        971  ...            431        200   
3                 0        4        891  ...            320         95   
4                 0        5       1619  ...            473        100   

   보건범죄_보건범죄  환경범죄_환경범죄  교통범죄_교통범죄  노동범죄_노동범죄  안보범죄_안보범죄  선거범죄_선거범죄  \
0        119 

In [121]:
# ----------------------------------------------------
# 2-1. 세부 범죄를 의미별로 묶어서 컬럼 재구성
#      (기존 세부 컬럼은 버리고, 묶인 컬럼만 남긴다)
# ----------------------------------------------------
crime_group_map = {
    # --- 강력범죄 ---
    "강력범죄_살인계": [
        "강력범죄_살인기수",
        "강력범죄_살인미수등",
    ],
    "강력범죄_강도": [
        "강력범죄_강도",
    ],
    "강력범죄_성범죄": [
        "강력범죄_강간",
        "강력범죄_유사강간",
        "강력범죄_강제추행",
    ],

    # --- 절도 ---
    "절도범죄_전체": [
        "절도범죄_절도범죄",
    ],

    # --- 폭력 (지금은 손괴만 있으니 그대로 전체로 사용) ---
    "폭력범죄_전체": [
        "폭력범죄_손괴",
    ],

    # --- 지능범죄: 재산 범죄 묶기 ---
    "지능범죄_재산범죄": [
        "지능범죄_사기",
        "지능범죄_횡령",
        "지능범죄_배임",
    ],

    # --- 풍속 ---
    "풍속범죄_전체": [
        "풍속범죄_성풍속범죄",
        "풍속범죄_도박범죄",
    ],

    # --- 나머지 대분류는 전체 하나로 ---
    "특별경제범죄_전체": ["특별경제범죄_특별경제범죄"],
    "마약범죄_전체":     ["마약범죄_마약범죄"],
    "보건범죄_전체":     ["보건범죄_보건범죄"],
    "환경범죄_전체":     ["환경범죄_환경범죄"],
    "교통범죄_전체":     ["교통범죄_교통범죄"],
    "노동범죄_전체":     ["노동범죄_노동범죄"],
    "안보범죄_전체":     ["안보범죄_안보범죄"],
    "선거범죄_전체":     ["선거범죄_선거범죄"],
    "병역범죄_전체":     ["병역범죄_병역범죄"],
    "기타범죄_전체":     ["기타범죄_기타범죄"],
}

# 새 DF에 지역 컬럼만 먼저 복사
grouped = pd.DataFrame()
grouped["지역"] = crime_pivot["지역"]

# 각 그룹별로 합계 계산해서 새 컬럼으로 추가
for new_col, cols in crime_group_map.items():
    existing_cols = [c for c in cols if c in crime_pivot.columns]

    if not existing_cols:
        print(f"[경고] {new_col}에 해당하는 원본 컬럼이 없음: {cols}")
        grouped[new_col] = 0
    else:
        grouped[new_col] = crime_pivot[existing_cols].sum(axis=1)

# 앞으로는 crime_pivot 대신 그룹화된 DF를 사용
crime_pivot = grouped

print("▶ 그룹화 후 crime_pivot columns:", crime_pivot.columns.tolist())
print(crime_pivot.head())


▶ 그룹화 후 crime_pivot columns: ['지역', '강력범죄_살인계', '강력범죄_강도', '강력범죄_성범죄', '절도범죄_전체', '폭력범죄_전체', '지능범죄_재산범죄', '풍속범죄_전체', '특별경제범죄_전체', '마약범죄_전체', '보건범죄_전체', '환경범죄_전체', '교통범죄_전체', '노동범죄_전체', '안보범죄_전체', '선거범죄_전체', '병역범죄_전체', '기타범죄_전체']
      지역  강력범죄_살인계  강력범죄_강도  강력범죄_성범죄  절도범죄_전체  폭력범죄_전체  지능범죄_재산범죄  풍속범죄_전체  \
0  서울종로구         3        5       239     1173      332       2878      142   
1   서울중구         3        6       216     1523      286       2919      129   
2  서울용산구         3        3       244      971      415       2156      137   
3  서울성동구         2        2       120      891      211       2398      152   
4  서울광진구         2        5       239     1619      375       2847      204   

   특별경제범죄_전체  마약범죄_전체  보건범죄_전체  환경범죄_전체  교통범죄_전체  노동범죄_전체  안보범죄_전체  선거범죄_전체  \
0        523      100      119        1      657        4       11        2   
1        727       73      108        0      652        3        4        3   
2        431      200      102        1     1076        4  

In [122]:
# ----------------------------------------------------
# 3. '지역'에서 행정구역(시) / 행정구역(시군구)별 분리
#    - 예: '서울종로구'      → 서울특별시 / 종로구
#          '경기도고양시'    → 경기도 / 고양시
#          '충북제천시'      → 충청북도 / 제천시
#          '제주제주시'      → 제주특별자치도 / 제주시
#          '세종시'          → 세종특별자치시 / 세종특별자치시
# ----------------------------------------------------
sido_prefixes = {
    "서울": "서울특별시",
    "부산": "부산광역시",
    "대구": "대구광역시",
    "인천": "인천광역시",
    "광주": "광주광역시",
    "대전": "대전광역시",
    "울산": "울산광역시",
    "세종": "세종특별자치시",
    "경기도": "경기도",
    "강원도": "강원도",
    "충북": "충청북도",
    "충남": "충청남도",
    "전북": "전라북도",
    "전남": "전라남도",
    "경북": "경상북도",
    "경남": "경상남도",
    "제주": "제주특별자치도",
}


In [123]:
def parse_region(col):
    """
    '서울종로구' → ('서울특별시', '종로구')
    '경기도고양시' → ('경기도', '고양시')
    '충북제천시'   → ('충청북도', '제천시')
    '제주제주시'   → ('제주특별자치도', '제주시')
    '세종시'       → ('세종특별자치시', '세종특별자치시')
    """
    raw = str(col).replace(" ", "")

    for prefix, sido in sido_prefixes.items():
        if raw.startswith(prefix):
            # 나머지가 시군구
            sgg = raw[len(prefix):]

            # 세종은 시 전체 = 시군구도 동일하게 처리
            if sido == "세종특별자치시":
                return sido, sido

            # 제주: '제주제주시', '제주서귀포시' 등
            if sido == "제주특별자치도":
                if sgg == "":
                    return sido, sido
                return sido, sgg

            return sido, sgg

    # prefix에 안 잡히는 경우 (이론상 거의 없음) → None 처리
    return None, None

# parse_region 적용해서 새 컬럼 두 개 생성
crime_pivot["행정구역(시)"], crime_pivot["행정구역(시군구)별"] = zip(
    *crime_pivot["지역"].apply(parse_region)
)


In [124]:
# 국내가 아닌 것(혹시 남아있다면) 제거
crime_pivot = crime_pivot.dropna(subset=["행정구역(시)", "행정구역(시군구)별"]).copy()

# 공백 제거 통일
crime_pivot["행정구역(시)"] = crime_pivot["행정구역(시)"].str.replace(" ", "", regex=False)
crime_pivot["행정구역(시군구)별"] = crime_pivot["행정구역(시군구)별"].str.replace(" ", "", regex=False)

# ----------------------------------------------------
# 4. 시 전체 합계 행 추가 (서울특별시 / 서울특별시, 부산광역시 / 부산광역시 ...)
# ----------------------------------------------------
# 범죄 컬럼들만 골라내기
crime_value_cols = [
    c for c in crime_pivot.columns
    if c not in ["지역", "행정구역(시)", "행정구역(시군구)별"]
]

# 시 단위 합계
city_totals = (
    crime_pivot
    .groupby("행정구역(시)")[crime_value_cols]
    .sum()
    .reset_index()
)
city_totals["행정구역(시군구)별"] = city_totals["행정구역(시)"]

# 전국 합계 (원하면 사용, 아니면 건너뛰어도 됨)
nation_total = city_totals[crime_value_cols].sum()
nation_row = pd.DataFrame([{
    "행정구역(시)": "전국",
    "행정구역(시군구)별": "전국",
    **{col: int(nation_total[col]) for col in crime_value_cols}
}])

# city_totals, nation_row 붙이기
crime_final = pd.concat(
    [crime_pivot, city_totals, nation_row],
    ignore_index=True
)

# 혹시 키가 중복되면 하나만 남기기 (안전용)
crime_final = (
    crime_final
    .drop_duplicates(subset=["행정구역(시)", "행정구역(시군구)별"], keep="first")
    .reset_index(drop=True)
)

print("▶ crime_final head:")
print(crime_final[["행정구역(시)", "행정구역(시군구)별"] + crime_value_cols[:5]].head())
print("crime_final shape:", crime_final.shape)


▶ crime_final head:
  행정구역(시) 행정구역(시군구)별  강력범죄_살인계  강력범죄_강도  강력범죄_성범죄  절도범죄_전체  폭력범죄_전체
0   서울특별시        종로구         3        5       239     1173      332
1   서울특별시         중구         3        6       216     1523      286
2   서울특별시        용산구         3        3       244      971      415
3   서울특별시        성동구         2        2       120      891      211
4   서울특별시        광진구         2        5       239     1619      375
crime_final shape: (247, 20)


In [132]:
merged_df[merged_df['행정구역(시군구)별']=='제주시']

,시점,행정구역(시),행정구역(시군구)별,총인구수,남자인구수,여자인구수,영아기,아동기,청소년기,청년기,...,특별경제범죄_전체,마약범죄_전체,보건범죄_전체,환경범죄_전체,교통범죄_전체,노동범죄_전체,안보범죄_전체,선거범죄_전체,병역범죄_전체,기타범죄_전체
260,2023,제주특별자치도,제주시,489858.5,244666.0,245192.5,15731.0,24137.5,52898.5,118246.0,...,700.0,84.0,42.0,13.0,3200.0,31.0,1.0,6.0,119.0,3275.0


In [126]:
# ----------------------------------------------------
# 5. area_df와 병합
#    키: ["행정구역(시)", "행정구역(시군구)별"]
# ----------------------------------------------------
merged_df = area_df.merge(
    crime_final,
    on=["행정구역(시)", "행정구역(시군구)별"],
    how="left"
)

print("▶ merged_df shape:", merged_df.shape)
print(merged_df.head())

# 이제 merged_df 안에:
# - 인구 / 연령대 비율 / 1인가구비율 / CCTV수 / 경찰서수 ...
# - + 각 범죄대분류_범죄중분류 별 발생건수 컬럼이 모두 들어 있음.

▶ merged_df shape: (261, 33)
     시점 행정구역(시) 행정구역(시군구)별       총인구수      남자인구수      여자인구수       영아기  \
0  2023   서울특별시      서울특별시  9334933.5  4517716.5  4817217.0  220265.5   
1  2023   서울특별시        강남구   529711.5   253227.5   276484.0   11753.0   
2  2023   서울특별시        강동구   456583.5   223164.5   233419.0   13400.5   
3  2023   서울특별시        강북구   288656.5   139838.5   148818.0    4806.0   
4  2023   서울특별시        강서구   562962.0   269867.5   293094.5   13845.5   

        아동기      청소년기        청년기  ...  특별경제범죄_전체  마약범죄_전체  보건범죄_전체  환경범죄_전체  \
0  308678.0  739051.5  2785782.5  ...    13181.0   3722.0   2850.0     46.0   
1   20850.0   60724.5   143489.0  ...     1663.0    972.0    689.0      6.0   
2   18305.0   38867.0   125691.5  ...      626.0     76.0     52.0      1.0   
3    7356.5   19546.5    74110.0  ...      342.0     93.0     58.0      1.0   
4   18529.5   42348.0   177448.5  ...      491.0    135.0    113.0      6.0   

   교통범죄_전체 노동범죄_전체  안보범죄_전체  선거범죄_전체  병역범죄_전체  기타범죄_전체  


In [127]:
merged_df = merged_df.drop(columns=["지역"])
merged_df = merged_df[merged_df["행정구역(시군구)별"] != "세종시"]

### **범죄율계산**

In [129]:
print("merged_df columns:", merged_df.columns.tolist())

merged_df columns: ['시점', '행정구역(시)', '행정구역(시군구)별', '총인구수', '남자인구수', '여자인구수', '영아기', '아동기', '청소년기', '청년기', '중년기', '노년기', 'CCTV', '1인가구비율', '경찰서수', '강력범죄_살인계', '강력범죄_강도', '강력범죄_성범죄', '절도범죄_전체', '폭력범죄_전체', '지능범죄_재산범죄', '풍속범죄_전체', '특별경제범죄_전체', '마약범죄_전체', '보건범죄_전체', '환경범죄_전체', '교통범죄_전체', '노동범죄_전체', '안보범죄_전체', '선거범죄_전체', '병역범죄_전체', '기타범죄_전체']


In [133]:
# 1) 범죄 컬럼 리스트 만들기
crime_cols = [c for c in merged_df.columns if "범죄" in c]

# 2) 각 범죄 컬럼을 인구 1,000명당 값으로 변환
for col in crime_cols:
    merged_df[col] = (merged_df[col] / merged_df["총인구수"]) * 1000


In [134]:
merged_df.head()

,시점,행정구역(시),행정구역(시군구)별,총인구수,남자인구수,여자인구수,영아기,아동기,청소년기,청년기,...,특별경제범죄_전체,마약범죄_전체,보건범죄_전체,환경범죄_전체,교통범죄_전체,노동범죄_전체,안보범죄_전체,선거범죄_전체,병역범죄_전체,기타범죄_전체
0,2023,서울특별시,서울특별시,9334933.5,4517716.5,4817217.0,220265.5,308678.0,739051.5,2785782.5,...,1.412008,0.398717,0.305305,0.004928,3.217591,0.007284,0.011141,0.005892,0.162186,5.266026
1,2023,서울특별시,강남구,529711.5,253227.5,276484.0,11753.0,20850.0,60724.5,143489.0,...,3.139445,1.834961,1.300708,0.011327,5.642694,0.033981,0.007551,0.005663,0.234090,9.469305
2,2023,서울특별시,강동구,456583.5,223164.5,233419.0,13400.5,18305.0,38867.0,125691.5,...,1.371053,0.166454,0.113889,0.002190,2.288738,0.000000,0.004380,0.004380,0.137981,3.541521
3,2023,서울특별시,강북구,288656.5,139838.5,148818.0,4806.0,7356.5,19546.5,74110.0,...,1.184799,0.322182,0.200931,0.003464,3.207965,0.006929,0.006929,0.003464,0.183609,6.190749
4,2023,서울특별시,강서구,562962.0,269867.5,293094.5,13845.5,18529.5,42348.0,177448.5,...,0.872173,0.239803,0.200724,0.010658,2.291451,0.003553,0.003553,0.021316,0.133224,4.362639


In [139]:
merged_df.isna().sum()

시점             0
행정구역(시)        0
행정구역(시군구)별     0
총인구수           0
남자인구수          0
여자인구수          0
영아기            0
아동기            0
청소년기           0
청년기            0
중년기            0
노년기            0
CCTV           0
1인가구비율         0
경찰서수           0
강력범죄_살인계      16
강력범죄_강도       16
강력범죄_성범죄      16
절도범죄_전체       16
폭력범죄_전체       16
지능범죄_재산범죄     16
풍속범죄_전체       16
특별경제범죄_전체     16
마약범죄_전체       16
보건범죄_전체       16
환경범죄_전체       16
교통범죄_전체       16
노동범죄_전체       16
안보범죄_전체       16
선거범죄_전체       16
병역범죄_전체       16
기타범죄_전체       16
dtype: int64

In [140]:
# 1) 범죄 컬럼 자동 추출
crime_cols = [c for c in merged_df.columns if "범죄" in c]

# 2) 각 범죄 컬럼의 NaN을 시 단위 평균으로 채우기
for col in crime_cols:
    merged_df[col] = merged_df.groupby("행정구역(시)")[col].transform(
        lambda x: x.fillna(x.mean())
    )


In [141]:
merged_df[crime_cols].isna().sum()

강력범죄_살인계     0
강력범죄_강도      0
강력범죄_성범죄     0
절도범죄_전체      0
폭력범죄_전체      0
지능범죄_재산범죄    0
풍속범죄_전체      0
특별경제범죄_전체    0
마약범죄_전체      0
보건범죄_전체      0
환경범죄_전체      0
교통범죄_전체      0
노동범죄_전체      0
안보범죄_전체      0
선거범죄_전체      0
병역범죄_전체      0
기타범죄_전체      0
dtype: int64

In [142]:
# 저장 경로 설정
save_path = "../../data/2.processed/processed_final.csv"

# CSV 저장
merged_df.to_csv(save_path, index=False, encoding='UTF-8')

print("저장 완료:", save_path)

저장 완료: ../../data/2.processed/processed_final.csv
